## Introduction

This kernel shows how to use NBSVM (Naive Bayes - Support Vector Machine) to create a strong baseline. In this kernel, we use sklearn's logistic regression, rather than SVM, although in practice the two are nearly identical (sklearn uses the liblinear library behind the scenes).

In [1]:
import pandas as pd, numpy as np
from sklearn.linear_model import LogisticRegression
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import re, string

## Take Input

In [2]:
# read the train and test data
def readInputFiles(train_file_path, test_file_path):
    train = pd.read_csv(train_file_path)
    test = pd.read_csv(test_file_path)
    return train, test
    
train, test = readInputFiles('../dataset/train_new.csv', '../dataset/test_new.csv')

## Data Exploration

In [3]:
def dataExploration(trainData):   
    # look at the top five rows of the train data file
    # training data contains a row per comment, with an id, 
    # the text of the comment, and 6 different labels
    trainData.head()
    
    # exampel of a comment in the train data
    print (trainData['comment_text'][0])
    print ("")
    
    # the length of the comments varies a lot   
    lens = trainData.comment_text.str.len()
    print("Average length of comments: %d\nStandard dev: %d\nLongest Comment: %d" % (lens.mean(), lens.std(), lens.max()))
    
    print ("")
    print ("# Comments in train data: %d \n# Comments in test data: %d" % (len(train),len(test)))
    lens.hist()
    
dataExploration(train)
train.head()

Grandma Terri Should Burn in Trash 
Grandma Terri is trash. I hate Grandma Terri. F%%K her to HELL! 71.74.76.40

Average length of comments: 395
Standard dev: 594
Longest Comment: 5895

# Comments in train data: 127656 
# Comments in test data: 31915


,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,140030,ed56f082116dcbd0,Grandma Terri Should Burn in Trash \nGrandma T...,1,0,0,0,0,0
1,159124,f8e3cd98b63bf401,", 9 May 2009 (UTC)\nIt would be easiest if you...",0,0,0,0,0,0
2,60006,a09e1bcf10631f9a,"""\n\nThe Objectivity of this Discussion is dou...",0,0,0,0,0,0
3,65432,af0ee0066c607eb8,Shelly Shock\nShelly Shock is. . .( ),0,0,0,0,0,0
4,154979,b734772b1a807e09,I do not care. Refer to Ong Teng Cheong talk p...,0,0,0,0,0,0


## Preprocessing Data

In [8]:
COMMENT = 'comment_text'

def preProcessData(trainData, testData):
    # create a list of all the labels to predict
    label_cols = ['toxic', 'severe_toxic', 'obscene', 'threat', 'insult', 'identity_hate']
    
    # create a 'none' label so we can see how many comments have no labels
    train['none'] = 1-train[label_cols].max(axis=1)
    
    train.describe()
    
    # get rid of the empty comments, otherwise sklearn complains
    train[COMMENT].fillna("unknown", inplace=True)
    test[COMMENT].fillna("unknown", inplace=True)
    return label_cols

label_cols = preProcessData(train, test)
train.head()

,Unnamed: 0,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate,none
0,140030,ed56f082116dcbd0,Grandma Terri Should Burn in Trash \nGrandma T...,1,0,0,0,0,0,0
1,159124,f8e3cd98b63bf401,", 9 May 2009 (UTC)\nIt would be easiest if you...",0,0,0,0,0,0,1
2,60006,a09e1bcf10631f9a,"""\n\nThe Objectivity of this Discussion is dou...",0,0,0,0,0,0,1
3,65432,af0ee0066c607eb8,Shelly Shock\nShelly Shock is. . .( ),0,0,0,0,0,0,1
4,154979,b734772b1a807e09,I do not care. Refer to Ong Teng Cheong talk p...,0,0,0,0,0,0,1


## Feature Extraction

In [9]:
def tokenize(s): 
    re_tok = re.compile(f'([{string.punctuation}“”¨«»®´·º½¾¿¡§£₤‘’])')
    return re_tok.sub(r' \1 ', s).split()

# create bag of words representation, as a term document matrix using ngrams
def wordRepresentation(trainData, testData):
    # TF-IDF gives even better priors than the binarized features. 
    # it improves leaderboard score from 0.59 to 0.55.
    vec = TfidfVectorizer(ngram_range=(1,2), tokenizer=tokenize,
                   min_df=3, max_df=0.9, strip_accents='unicode', use_idf=1,
                   smooth_idf=1, sublinear_tf=1 )

    # Extracting features from the training data using a sparse vectorizer"
    train_term_doc = vec.fit_transform(trainData[COMMENT])

    # Extracting features from the test data using the same vectorizer
    test_term_doc = vec.transform(testData[COMMENT])

    # a sparse matrix with only a small number of non-zero elements with the below shape
    print("n_comments_train: %d, n_features: %d" % train_term_doc.shape)
    print("n_comments_test: %d, n_features: %d" % test_term_doc.shape)
    
    return train_term_doc, test_term_doc, vec
    
train_term_doc, test_term_doc, vec = wordRepresentation(train, test)

n_comments_train: 127656, n_features: 64631
n_comments_test: 31915, n_features: 64631


## Basic Naive Bayes Model

In [10]:
def pr(y_i, y):
    p = x[y==y_i].sum(0)
    return (p+1) / ((y==y_i).sum()+1)

In [11]:
x = train_term_doc
test_x = test_term_doc

In [12]:
# Fit a model for one dependent at a time
def get_mdl(y):
    y = y.values
    r = np.log(pr(1,y) / pr(0,y))
    m = LogisticRegression(C=4, dual=True)
    x_nb = x.multiply(r)
    return m.fit(x_nb, y), r

In [13]:
# fit model and make predictions
def getPredictions(test, label_cols):
    
    model_r_dict = {}
    
    preds = np.zeros((len(test), len(label_cols)))

    for i, j in enumerate(label_cols):
        print('fitting', j)
        m,r = get_mdl(train[j])
        
        # store model and r value for future predictions
        model_r_dict[i] = r, m 
        
        # get predictions
        preds[:,i] = m.predict_proba(test_x.multiply(r))[:,1]

    return preds, model_r_dict
    
preds, model_r_dict = getPredictions(test, label_cols)


('fitting', 'toxic')
('fitting', 'severe_toxic')
('fitting', 'obscene')
('fitting', 'threat')
('fitting', 'insult')
('fitting', 'identity_hate')


## Save Results to a file

In [14]:
# create the submission file and store the results
def saveResults(predictions):
    submission = pd.DataFrame.from_dict({'id': test['id']})
    for idx, col in enumerate(label_cols):
        submission[col] = predictions[:,idx]
    submission.to_csv('submission_new.csv', index=False)
    
saveResults(preds)

In [22]:
from sklearn.metrics import log_loss
import sys, os, re, csv, codecs, numpy as np, pandas as pd

def calc_loss(y_true, y_pred):
    return np.mean([log_loss(y_true[:, i], y_pred[:, i]) 
                    for i in range(y_true.shape[1])])


In [29]:
from sklearn.metrics import roc_auc_score
def auc_score(y_true, y_pred):
    return np.mean([roc_auc_score(y_true[:, i], y_pred[:, i]) 
                    for i in range(y_true.shape[1])])
    
    

In [30]:
true = pd.read_csv('../dataset/test_new.csv')
pred = pd.read_csv('submission_new.csv')

list_classes = ["toxic", "severe_toxic", "obscene", "threat", "insult", "identity_hate"]
y_true = true[list_classes].values
y_pred = pred[list_classes].values

loss = calc_loss(y_true, y_pred)
print ("Log Loss = ", loss)
auc = auc_score(y_true, y_pred)
print ("AUC = ", auc)

('Log Loss = ', 0.07104368821910462)
('AUC = ', 0.9510252031051949)


## Demo

In [33]:
# for demo purposes only
def demo(vectorizer, label_cols, model_r_dict):
    testing_comment = input("Enter a comment: ")

    # testing_comment = ["Yo bitch Ja Rule is more succesful then you'll ever be whats up with you and hating you sad mofuckas...i should bitch slap ur pethedic white faces and get you to kiss my ass you guys sicken me. Ja rule is about pride in da music man. dont diss that shit on him. and nothin is wrong bein like tupac he was a brother too...fuckin white boys get things right next time."]

    # vectorizer.transform takes list input so pass a list
    user_comment = []
    user_comment.append(testing_comment)
    
    # Extracting features from the test data using the vectorizer
    test_data_x = vectorizer.transform(user_comment)
    
    # to store the predictions
    prediction = np.zeros((1, len(label_cols)))

    # make prediction using the model created
    for i, j in enumerate(label_cols):
        r = model_r_dict[i][0]
        m = model_r_dict[i][1]
        prediction[0,i] = m.predict_proba(test_data_x.multiply(r))[0,1]
    
    # copy the result and display
    pred_y = pd.DataFrame(columns=label_cols)
    for idx, col in enumerate(label_cols):
        pred_y.at[0, col] = prediction[0,idx]  
     
    print ("")
    print (pred_y)
#     # The String I used above is for the first comment, that is, result of loc 0 in submission file
#     print ("This is the actual result: ", submission.loc[0])
    
demo(vec, label_cols, model_r_dict)      

Enter a comment: "F U C !"

     toxic severe_toxic   obscene       threat    insult identity_hate
0  0.98773    0.0938766  0.995949  0.000496029  0.780581    0.00598765
